生成约束和变量个数信息

In [ ]:
import pandas as pd
import os
import re
import tqdm
import matplotlib.pyplot as plt

class QF_NRA:
    count = 0
    def __init__(self, init=False, path='QF_NRA_spilt', data_path='result'):
        self.path = path
        self.data_path = data_path
        if init:
            self.get_info(path)
        self.records = pd.read_excel(r"base.xlsx", header=[0], index_col = [0])

    def get_info(self):
        folder_list = os.listdir(self.path)
        for folder in folder_list:
            new_path = os.path.join(self.path, folder)
            if os.path.isdir(new_path):
                self.get_info(new_path)
            else:
                with open(new_path) as f:
                    vars = 0
                    assertions = 0
                    line = f.readline()
                    while line:
                        if line[:3] == '(de':
                            vars += 1
                        elif line[:3] == '(as':
                            assertions += 1
                        line = f.readline()
                    # print(count, "/", total,": ", filename, "+", vars, "+", assertions)
                    self.count += 1
                    category = new_path.split('/')[1]
                    self.records = self.records.append({'filename': new_path, 'category':category, '#vars':vars, '#assertions':assertions}, ignore_index=True, verify_integrity=True)    
        
    def arrange(self):
        file_list = os.listdir(self.data_path)
        for filename in tqdm(file_list, desc = self.data_path):
            with open(os.path.join(self.data_path, filename)) as f:
                while f.readline():
                    result = f.readline()
                    if result[:3] == 'run':
                        result = 'dump'
                    info = f.readline()
                    name = info.split(' : ')[0]
                    timeout = info.split(' : ')[1][:-5]
                    self.records.loc[self.records['filename'] == name, ['result', 'timeout']] = [result, timeout]
        
    def save_to(self, filename):
        self.records.to_excel(filename)
    
    def show(self):
        folder_list = os.listdir(self.path)
        name = []
        solved = []
        count = []
        dump = []
        timeout = []
        for folder in folder_list:
            name.append(folder)
            count.append(len(self.records[self.records['category'] == folder]))
            solved.append(len(self.records[self.records['category'] == folder & self.records['result'].str.contains('sat')]))
            dump.append(len(self.records[self.records['category'] == folder & self.records['result'] == ('dump')]))
            timeout.append(len(self.records[self.records['category'] == folder & self.records['result'] == ('timeout')])

        plt.bar(folder_list, name, label="name")
        plt.bar(folder_list, solved, label="solved")
        plt.bar(folder_list, count, label="count")
        plt.bar(folder_list, timeout, label="timeout")
        plt.show()

In [2]:
new_path = 'QF_NRA_split/20161105-Sturm-MBO_2/mbo_E3E11.smt2'
category = new_path.split('/')[1]
category

'20161105-Sturm-MBO_2'